In [ ]:

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os
import json
import csv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)
save_dir = os.getcwd()
print("Saving data to:", save_dir)
num_trials = 1

steady_check_interval = 10   
steady_threshold = 1e-5      
avg_window = 40              

def check_steady(kinetic_history, window=3, threshold=1e-5):  #判断是否达到稳态
    if len(kinetic_history) < 2 * window:
        return False
    recent = np.mean(kinetic_history[-window:])
    prev = np.mean(kinetic_history[-2*window:-window])
    return abs(recent - prev) < threshold

def sample_boltzmann(J,X0, T, num_trials=num_trials):
    traj = []
    for _ in range(num_trials):
        kinetic_history = []
        buffer = []
        steady_reached = False    
        x = X0.clone()
        for t in range(steps):
            x = x.detach().requires_grad_(True)
            inner = -x + J @ phi(x)
            E = 0.5 * (inner**2).sum()
            gradE, = torch.autograd.grad(E, x, create_graph=False)
            noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
            dx = -gradE * dt + noise
            x = x + dx
            buffer.append((dx**2).mean().item()/2)
            if (t + 1) % 10 == 0:
                mean_kin = np.mean(buffer[-10:])
                kinetic_history.append(mean_kin)
                if t>2000 and check_steady(kinetic_history, threshold=1e-5):
                    steady_reached = True
                    # print(f"Grad-descent Steady state reached after {t} steps")
                    break

        if steady_reached:
            for j in range(M*10):
                x = x.detach().requires_grad_(True)
                inner = -x + J @ phi(x)
                E = 0.5 * (inner**2).sum()
                gradE, = torch.autograd.grad(E, x, create_graph=False)

                noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
                dx = -gradE * dt + noise
                x = x + dx
                if j % 10 == 0:
                    traj.append(x.detach().cpu().numpy())
    return np.array(traj)

def Classical_sample_energy(J,X0, num_trials=num_trials):
    samples_rnn = []
    steady_reached = False
    for _ in range(num_trials):
        # torch.manual_seed(1997)
        # x = torch.randn(N, device=device)
        x = X0.clone()
        kinetic_history = []
        buffer = []
        steady_reached = False
        for t in range(steps):
            dx = -x + J @ phi(x)
            x = x + dt * dx
            buffer.append((dx**2).mean().item()/2)
            if (t + 1) % 10 == 0:
                mean_kin = np.mean(buffer[-10:])
                kinetic_history.append(mean_kin)
                if t>2000 and check_steady(kinetic_history, threshold=1e-5):
                    steady_reached = True
                    # print(f"RNN Steady state reached after {t} steps")
                    break
        if  steady_reached:
            for j in range(M*10):  
                dx = -x + J @ phi(x)
                x = x + dt * dx
                if j % 10 == 0:
                    samples_rnn.append(x.detach().cpu().numpy())
    return np.array(samples_rnn)


phi = torch.tanh

def compute_center_and_angles(samples_np):
    """return: phi_chosen (m,N), center c (N,), angles psi (m,) (radians)"""
    if samples_np is None or len(samples_np) == 0:
        return np.zeros((0,)), np.zeros((0,)), np.zeros((0,))
    chosen_torch = torch.tensor(samples_np, device=device, dtype=torch.float32)
    phi_chosen_torch = phi(chosen_torch)
    phi_chosen = phi_chosen_torch.cpu().numpy()
    c = np.mean(phi_chosen, axis=0)  # (N,)
    dot = phi_chosen @ c  # (m,)
    norm_phi = np.linalg.norm(phi_chosen, axis=1)  # (m,)
    norm_c = np.linalg.norm(c)  # scalar# 避免除以 0
    denom = norm_phi * (norm_c + 1e-16)
    cos_vals = np.zeros_like(dot)
    nonzero_mask = denom > 0
    cos_vals[nonzero_mask] = dot[nonzero_mask] / denom[nonzero_mask]
    cos_vals = np.clip(cos_vals, -1.0, 1.0)
    psi = np.arccos(cos_vals)  # radians
    return psi,c,norm_phi

def angle_between_vectors(u, v):
    if u is None or v is None or len(u)==0 or len(v)==0:
        return np.nan
    dot = np.dot(u, v)
    nu = np.linalg.norm(u)
    nv = np.linalg.norm(v)
    denom = (nu * nv) + 1e-16
    cosv = np.clip(dot / denom, -1.0, 1.0)
    return np.arccos(cosv)  # radians


output_file_raw_1= "AngleCompare_N_new3.csv"
if not os.path.exists(output_file_raw_1):
    df_init = pd.DataFrame(columns=["T","N","theta_deg","psi_R_deg","psi_G_deg"])
    df_init.to_csv(output_file_raw_1, index=False, quoting=csv.QUOTE_ALL)

M=3000
steps = 25000
dt = 0.1
g=1.5
df = pd.read_csv("Teff.csv")
Te = df['T_eff']

NList = df['N']
print (NList)
print(Te)
all_results = []
results = []
for i in range(len(Te)-1):
    T1 = Te[i]
    N = NList[i]
    torch.manual_seed(2026)
    J = torch.randn(N, N, device=device) * g / np.sqrt(N)
    X0 = torch.randn(N, device=device)

    samples_boltz = sample_boltzmann(J, X0,T1)
    samples_classi = Classical_sample_energy(J,X0, num_trials=num_trials)
    print (samples_boltz.shape)

    psi_R, c_R,norm_R = compute_center_and_angles(samples_classi)
    psi_G, c_G,norm_G = compute_center_and_angles(samples_boltz)
    # center angle
    dot = np.dot(c_R, c_G)
    theta = np.arccos(dot / (np.linalg.norm(c_R)*np.linalg.norm(c_G)+1e-16))
    theta_deg = np.degrees(theta)
    norm_c_G = np.linalg.norm(c_G)
    norm_c_R = np.linalg.norm(c_R)

    psi_R_deg = np.degrees(psi_R)
    psi_G_deg = np.degrees(psi_G)
    if N == 2000:
        results.append({
            "T": T1,
            "N": N,
            "theta_deg": theta_deg,
            "psi_R_deg": json.dumps(psi_R_deg.tolist()),
            "psi_G_deg": json.dumps(psi_G_deg.tolist()),
            "norm_phi_R": json.dumps(norm_R.tolist()),
            "norm_phi_G": json.dumps(norm_G.tolist()),
            'norm_c_R':norm_c_R,
            'norm_c_G':norm_c_G,
        })

output_file_raw = "AngleCompare_diffM_new3.csv"
if not os.path.exists(output_file_raw):
    df_init = pd.DataFrame(columns=["T","M","theta_deg","psi_R_deg","psi_G_deg"])
    df_init.to_csv(output_file_raw, index=False, quoting=csv.QUOTE_ALL)


steps = 250000
results_M = []
M_list = np.linspace(1000, 21000, 11, dtype=int).tolist()
# torch.manual_seed(1997)
torch.manual_seed(2026)
N = 5000
J = torch.randn(N, N, device=device) * g / np.sqrt(N)
X = torch.randn(N, device=device)
for M in tqdm(M_list, desc="Scanning M"):
    print(f"M={M}")
    T1 = Te[1]
    X0 = X.clone()  
    samples_boltz = sample_boltzmann(J, X0,T1)
    samples_classi = Classical_sample_energy(J,X0, num_trials=num_trials)
    print (samples_boltz.shape)

    psi_R, c_R,norm_R = compute_center_and_angles(samples_classi)
    psi_G, c_G,norm_G = compute_center_and_angles(samples_boltz)
    # center angle
    dot = np.dot(c_R, c_G)
    theta = np.arccos(dot / (np.linalg.norm(c_R)*np.linalg.norm(c_G)+1e-16))
    theta_deg = np.degrees(theta)
    psi_R_deg = np.degrees(psi_R)
    psi_G_deg = np.degrees(psi_G)
    norm_c_G = np.linalg.norm(c_G)
    norm_c_R = np.linalg.norm(c_R)

    results_M.append({
        "T": T1,
        "M": M,
        "theta_deg": theta_deg,
        "psi_R_deg": json.dumps(psi_R_deg.tolist()),
        "psi_G_deg": json.dumps(psi_G_deg.tolist()),
        "norm_phi_R": json.dumps(norm_R.tolist()),
        "norm_phi_G": json.dumps(norm_G.tolist()),
        'norm_c_R':norm_c_R,
        'norm_c_G':norm_c_G,
    })

    if M == 3000:
        results.append({
            "T": T1,
            "N": N,
            "theta_deg": theta_deg,
            "psi_R_deg": json.dumps(psi_R_deg.tolist()),
            "psi_G_deg": json.dumps(psi_G_deg.tolist()),
            "norm_phi_R": json.dumps(norm_R.tolist()),
            "norm_phi_G": json.dumps(norm_G.tolist()),
            'norm_c_R':norm_c_R,
            'norm_c_G':norm_c_G,
        })

df_save_M = pd.DataFrame(results_M)
df_save_M.to_csv(output_file_raw, index=False, quoting=csv.QUOTE_ALL)
print(f"\nAll results saved to {output_file_raw}")

df_save = pd.DataFrame(results)
df_save.to_csv(output_file_raw_1, index=False, quoting=csv.QUOTE_ALL)
print(f"\nAll results saved to {output_file_raw_1}")